In [7]:
from pykrx import stock
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Define the period for the simulation
start_date = '20230401'
end_date = '20240401'

# Fetch the KOSPI top 40 stocks by market cap (example tickers for simulation)
tickers = stock.get_market_ticker_list(date=end_date, market="KOSPI")[:40]

# Download stock data
dfs = []
for ticker in tickers:
    df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
    df['Ticker'] = ticker
    dfs.append(df)

data = pd.concat(dfs)
data = data.reset_index().pivot('index', 'Ticker', '종가')

# Calculate daily returns
returns = data.pct_change().dropna()

# Calculate mean returns and covariance matrix
mean_returns = returns.mean()
cov_matrix = returns.cov()

# Number of portfolios to simulate
num_portfolios = 50000

# Set up array to hold results
results = np.zeros((4, num_portfolios))

# Function to calculate portfolio performance
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights) * 252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

# Function to minimize (negative Sharpe ratio)
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    std, returns = portfolio_performance(weights, mean_returns, cov_matrix)
    return - (returns - risk_free_rate) / std

# Constraints
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(len(tickers)))

# Random portfolio generation
for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)
    std, ret = portfolio_performance(weights, mean_returns, cov_matrix)
    sharpe_ratio = (ret - 0.01) / std
    results[0,i] = ret
    results[1,i] = std
    results[2,i] = sharpe_ratio
    results[3,i] = np.sum(weights)

# Optimal portfolio
opt_sharpe = minimize(neg_sharpe_ratio, len(tickers)*[1./len(tickers)], args=(mean_returns, cov_matrix),
                      method='SLSQP', bounds=bounds, constraints=constraints)

# Efficient frontier
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_performance(weights, mean_returns, cov_matrix)[0]

frontier_y = np.linspace(0, 0.3, 100)
frontier_x = []

for ret in frontier_y:
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': lambda x: portfolio_performance(x, mean_returns, cov_matrix)[1] - ret}
    )
    result = minimize(portfolio_volatility, len(tickers)*[1./len(tickers)], args=(mean_returns, cov_matrix),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    frontier_x.append(result['fun'])

# Plot results
plt.figure(figsize=(10, 7))
plt.scatter(results[1,:], results[0,:], c=results[2,:], cmap='YlGnBu', marker='o')
plt.scatter(portfolio_performance(opt_sharpe['x'], mean_returns, cov_matrix)[0],
            portfolio_performance(opt_sharpe['x'], mean_returns, cov_matrix)[1], marker='*', color='r', s=500)
plt.plot(frontier_x, frontier_y, linestyle='--', color='black', marker='.')
plt.title('Efficient Frontier')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.colorbar(label='Sharpe Ratio')
plt.show()


ModuleNotFoundError: No module named 'pykrx'

In [1]:
from pykrx import stock

# Example: Get a list of KOSPI tickers
tickers = stock.get_market_ticker_list(market="KOSPI")
print(tickers)

ModuleNotFoundError: No module named 'pykrx'